In [ ]:
try:
    import import_ipynb
except :
    !pip install import_ipynb
    import import_ipynb
import Predistim_Library as pl

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
  Obtaining dependency information for import_ipynb from https://files.pythonhosted.org/packages/ec/62/e0b830773060d2a390aa923dcc8afc680d798bdbdadb6394f760fac62517/import_ipynb-0.2-py3-none-any.whl.metadata
